<a href="https://colab.research.google.com/github/bhas10bc/Deep_learning/blob/main/titanic_survival_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [146]:
# import the dependencies

import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [147]:
# load the datasets

train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [148]:
# Preprocessing the data
def preprocess_data(data):
  #Drop irrelevant column
  data = data.drop(['Name','Ticket','Cabin'], axis=1)

  # Fill missing values
  data['Age'].fillna(data['Age'].mean(), inplace=True)
  data['Fare'].fillna(data['Fare'].mean(), inplace=True)
  data['Embarked'].fillna(data['Embarked'].mode()[0], inplace=True)

  # convert categorical data to numerical
  data['Sex'] = LabelEncoder().fit_transform(data['Sex'])
  data['Embarked'] =  LabelEncoder().fit_transform(data['Embarked'])

  return data

In [149]:
train_data = preprocess_data(train_data)
test_data = preprocess_data(test_data)

In [150]:
X = train_data.drop('Survived', axis =1)
y = train_data['Survived']

In [151]:
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size=0.2, random_state=42)

In [152]:
X_train.shape

(712, 8)

In [153]:
# Feature scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

In [154]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(8,)),
    tf.keras.layers.Dense(64, activation='relu'),

    tf.keras.layers.Dense(32, activation='relu'),

    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

In [155]:
#compile the mopdel
model.compile(optimizer='adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

In [156]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs = 100, batch_size=32)

Epoch 1/100
23/23 [==============================] - 1s 10ms/step - loss: 0.6225 - accuracy: 0.7247 - val_loss: 0.5623 - val_accuracy: 0.8268
Epoch 2/100
23/23 [==============================] - 0s 4ms/step - loss: 0.5259 - accuracy: 0.8034 - val_loss: 0.4773 - val_accuracy: 0.8101
Epoch 3/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4624 - accuracy: 0.8090 - val_loss: 0.4324 - val_accuracy: 0.7989
Epoch 4/100
23/23 [==============================] - 0s 3ms/step - loss: 0.4342 - accuracy: 0.8090 - val_loss: 0.4162 - val_accuracy: 0.7989
Epoch 5/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4225 - accuracy: 0.8118 - val_loss: 0.4104 - val_accuracy: 0.8101
Epoch 6/100
23/23 [==============================] - 0s 4ms/step - loss: 0.4156 - accuracy: 0.8216 - val_loss: 0.4054 - val_accuracy: 0.8101
Epoch 7/100
23/23 [==============================] - 0s 3ms/step - loss: 0.4094 - accuracy: 0.8188 - val_loss: 0.4038 - val_accuracy: 0.8101
Epoch 8/100


In [157]:

X_test = test_data
X_test = scaler.transform(X_test)
test_predictions = model.predict(X_test)

14/14 [==============================] - 0s 2ms/step


In [158]:
test_predictions = (test_predictions > 0.5).astype(int)

In [159]:
from sklearn.metrics import roc_auc_score

# Assuming you have your model and test data prepared as previously mentioned
y_pred_test = model.predict(X_test)  # Use predictions on the test set

# Define y_test with the actual target values for the test set
y_test = pd.read_csv("gender_submission.csv")["Survived"]  # Load the actual test values from your submission file

# Calculate the ROC AUC score for the test data
roc_auc = roc_auc_score(y_test, y_pred_test)

print(f"ROC AUC Score on Test Data: {roc_auc}")


14/14 [==============================] - 0s 2ms/step
ROC AUC Score on Test Data: 0.9028739612188365


In [160]:
submission = pd.DataFrame({'PassengerId': test_data['PassengerId'], 'Survived': test_predictions.ravel()})

In [162]:
# Save the submission as a CSV file
submission.to_csv('titanic_submission.csv', index=False)